In [62]:
from numpy import array, asarray, zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import pandas as pd
import re
import keras
from sklearn.model_selection import train_test_split

In [63]:
# read input document
X = pd.read_csv('Sentiment.csv')
X.head(10)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
5,6,Ted Cruz,0.6332,yes,1.0000,Positive,0.6332,None of the above,1.0000,NaN,...,NaN,228,NaN,NaN,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",NaN,2015-08-07 09:54:44 -0700,629697194283499520,NaN,Central Time (US & Canada)
6,7,No candidate mentioned,1.0000,yes,1.0000,Negative,0.6761,FOX News or Moderators,1.0000,NaN,...,NaN,17,NaN,NaN,RT @warriorwoman91: I liked her and was happy ...,NaN,2015-08-07 09:54:44 -0700,629697192383672320,North Georgia,Eastern Time (US & Canada)
7,8,No candidate mentioned,1.0000,yes,1.0000,Neutral,1.0000,None of the above,1.0000,NaN,...,NaN,0,NaN,NaN,Going on #MSNBC Live with @ThomasARoberts arou...,NaN,2015-08-07 09:54:44 -0700,629697192169750528,New York NY,Eastern Time (US & Canada)
8,9,Ben Carson,1.0000,yes,1.0000,Negative,0.6889,None of the above,0.6444,NaN,...,NaN,0,NaN,NaN,Deer in the headlights RT @lizzwinstead: Ben C...,NaN,2015-08-07 09:54:44 -0700,629697190219243524,NaN,Pacific Time (US & Canada)
9,10,No candidate mentioned,0.4594,yes,0.6778,Negative,0.6778,None of the above,0.4594,NaN,...,NaN,1,NaN,NaN,RT @NancyOsborne180: Last night's debate prove...,NaN,2015-08-07 09:54:42 -0700,629697185093824512,NaN,NaN


In [64]:
X = X[['text', 'sentiment']]
X = X[X.sentiment != 'Neutral']
X['text'] = X['text'].apply(lambda x: x.lower())
X['text'] = X['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
print(X)

for idx, row in X.iterrows():
    row[0] = row[0].replace('rt', ' ')

print(X['sentiment'])

                                                    text sentiment
1      rt scottwalker didnt catch the full gopdebate ...  Positive
3      rt robgeorge that carly fiorina is trending  h...  Positive
4      rt danscavino gopdebate w realdonaldtrump deli...  Positive
5      rt gregabbott_tx tedcruz on my first day i wil...  Positive
6      rt warriorwoman91 i liked her and was happy wh...  Negative
...                                                  ...       ...
13866  rt cappy_yarbrough love to see men who will ne...  Negative
13867  rt georgehenryw who thought huckabee exceeded ...  Positive
13868  rt lrihendry tedcruz as president i will alway...  Positive
13869  rt jrehling gopdebate donald trump says that h...  Negative
13870  rt lrihendry tedcruz headed into the president...  Positive

[10729 rows x 2 columns]
1        Positive
3        Positive
4        Positive
5        Positive
6        Negative
           ...   
13866    Negative
13867    Positive
13868    Positive
13869   

In [65]:
# use tokenizer and pad
maxFeatures = 2000
tokenizer = Tokenizer(num_words=maxFeatures, split=' ')
tokenizer.fit_on_texts(X['text'].values)
encodeDocuments = tokenizer.texts_to_sequences(X['text'].values)

print(X['text'].values[0])
print(encodeDocuments[0])
print(tokenizer.index_word[341])

  scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
[363, 122, 1, 703, 2, 39, 58, 237, 37, 210, 6, 174, 1761, 12, 1324, 1409, 743]
saying


In [66]:
max_length = 29
paddedDocuments = pad_sequences(encodeDocuments, maxlen=max_length, padding='post')

# load glove model
inMemoryGlove = dict()
f = open('D:/AIdata/glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefficients = asarray(values[1:], dtype='float32')
    inMemoryGlove[word] = coefficients
f.close()
print(len(inMemoryGlove))

400000


In [68]:
# split data
labels = []
for i in X['sentiment']:
    if i == 'Positive':
        labels.append(1)
    # elif i == 'Negative':
    #     labels.append(-1)
    else:
        labels.append(0)

labels = array(labels)
# labels = keras.utils.to_categorical(labels,2)


X_train, X_test, Y_train, Y_test = train_test_split(paddedDocuments,labels, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 29) (7188,)
(3541, 29) (3541,)


In [69]:
# create coefficient matrix for training data
trainingToEmbeddings = zeros((maxFeatures, 100))
for word, i in tokenizer.word_index.items():
    if i < 2000:
        gloveVector = inMemoryGlove.get(word)
        if gloveVector is not None:
            trainingToEmbeddings[i] = gloveVector

In [70]:
model = Sequential()
model.add(Embedding(maxFeatures, 100, weights=[trainingToEmbeddings], input_length=max_length, trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 29, 100)           200000    
_________________________________________________________________
flatten_7 (Flatten)          (None, 2900)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 2901      
Total params: 202,901
Trainable params: 2,901
Non-trainable params: 200,000
_________________________________________________________________
None


In [71]:
batch_size = 32
model.fit(X_train, Y_train, epochs=50, batch_size=batch_size, verbose=0)

loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Accuracy: %f' % (accuracy * 100))

Accuracy: 80.965829
